Op verzoek van het RIVM worden enkele van de grotere bestanden voortaal 'opgesplitst' aangeleverd, d.w.z. in een eenmalig 'statisch' bestand met data van voor 4-10-2021 en in een dagelijks bestand dat de data bevat vanaf 4-10-2021. Deze opsplitsing is gewenst voor initieel 9 bestanden en op termijn zo'n 30 bestanden. 

Hier wordt dit allereerst gedaan voor een eerste groep van 9 bestanden. Eerst wordt er een lege tabel aangemaakt in het VWSSTATIC schema, zodat daarin eenmalig de statische gegevens kunnen worden ingelezen. Daarna wordt de stored procedure aangepast die de gegevens van VWSSTAGE naar VWSINTER kopieert. 

Bij de aanpassing van de stored procedure is van belang om goed om te gaan met de kolom Date\_last\_inserted.  In de VWSINTER tabellen wordt deze standaard automatisch gevuld met een tijdstempel via GetDate(). Dat gaat goed als er vanuit één enkele bron wordt ingelezen (die krijgt dan voor de hele bron dezelfde tijdstempel). Hier wordt echter ingelezen vanuit twee bronnen, in twee verschillende insert-operaties met elk een eigen tijdstempel. Zonder aanpassingen krijgen de twee bronnen twee verschillende tijdstempels. In de verdere verwerking wordt steeds gefilterd op de Max() van de Date\_Last\_Inserted en dat zou betekenen dat alleen de tweede insert wordt doorgegeven. Om dit te voorkomen is de automatische vulling van de kolom Date\_Last\_Inserted uitgezet, wordt voor de eerste insert de tijdstempel in een variabele gezet en wordt voor beide inserts deze variabele meegegeven als de bijbehorende tijdstempel.

De tabellen die zo worden aangepast zijn:

1. RIVM\_COVID\_19\_NUMBER\_MUNICIPALITY
    
2. DISABILITY
    
3. ELDERLY\_AT\_HOME
    
4. GGD\_TESTED\_PEOPLE
    
5. NURSING\_HOMES\_COMBINED
    
6. RIVM\_REPRODUCTION\_NUMBER
    
7. RIVM\_IC\_OPNAME
    
8. NICE\_HOSPITAL
    
9. RIVM\_HOSPITAL\_IC\_ADMISSIONS\_OVERTIME\_BYAGEGROUP

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

In [ ]:


IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_COVID_19_NUMBER_MUNICIPALITY'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_COVID_19_NUMBER_MUNICIPALITY](
		[ID] INT IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_PUBLICATION] [varchar](100) NULL,
		[MUNICIPALITY_CODE] [varchar](100) NULL,
		[MUNICIPALITY_NAME] [varchar](100) NULL,
		[PROVINCE] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
		[ROAZ_REGION] [varchar](100) NULL,
		[TOTAL_REPORTED] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[DECEASED] [varchar](100) NULL,
		[VERSION] [varchar](50) NULL,
		[DATE_LAST_INSERTED] DateTime Default GetDate()
	) ON [PRIMARY]
END
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'DISABILITY'))
BEGIN
	CREATE TABLE [VWSSTATIC].[DISABILITY](
		[ID] INT IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[Date_Last_Inserted] DateTime Default GetDate()
	) 
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'DISABILITY'))
BEGIN
	CREATE TABLE [VWSSTAGE].[DISABILITY](
		[ID] INT IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[Date_Last_Inserted] DateTime Default GetDate()
	) 
END

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_Disability_Inter]
AS
BEGIN

   -- Zet de Date_Last_Inserted in een variabele
    DECLARE @DateTimeNow AS DateTime = GETDATE()

    INSERT INTO VWSINTER.DISABILITY (
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        Date_Last_Inserted
        )
    SELECT
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        @DateTimeNow
    FROM
        VWSSTAGE.DISABILITY
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.DISABILITY)

    INSERT INTO VWSINTER.DISABILITY (
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        Date_Last_Inserted
        )
    SELECT
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        @DateTimeNow
    FROM
        VWSSTATIC.DISABILITY
    WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.DISABILITY)


END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'ELDERLY_AT_HOME'))
BEGIN

	CREATE TABLE [VWSSTATIC].[ELDERLY_AT_HOME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'ELDERLY_AT_HOME'))
BEGIN

	CREATE TABLE [VWSSTAGE].[ELDERLY_AT_HOME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_Elderly_At_Home_Inter]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.ELDERLY_AT_HOME (
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    Date_Last_Inserted
    )
SELECT
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    @DateTimeNow
FROM VWSSTAGE.ELDERLY_AT_HOME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.ELDERLY_AT_HOME)


INSERT INTO VWSINTER.ELDERLY_AT_HOME (
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    Date_Last_Inserted
    )
SELECT
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    @DateTimeNow
FROM VWSSTATIC.ELDERLY_AT_HOME
WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.ELDERLY_AT_HOME)


END;
GO


In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'GGD_TESTED_PEOPLE'))
BEGIN

	CREATE TABLE [VWSSTATIC].[GGD_TESTED_PEOPLE](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TESTED_WITH_RESULT] [varchar](100) NULL,
		[TESTED_POSITIVE] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END


In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'GGD_TESTED_PEOPLE'))
BEGIN

	CREATE TABLE [VWSSTAGE].[GGD_TESTED_PEOPLE](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TESTED_WITH_RESULT] [varchar](100) NULL,
		[TESTED_POSITIVE] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_GGD_TESTED_PEOPLE_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.GGD_TESTED_PEOPLE
    (
       [VERSION],
       DATE_OF_REPORT,
       DATE_OF_STATISTICS,
       SECURITY_REGION_CODE,
       SECURITY_REGION_NAME,
       TESTED_WITH_RESULT,
       TESTED_POSITIVE,
       Date_Last_Inserted
    )
    SELECT
        [VERSION],
        DATE_OF_REPORT,
        DATE_OF_STATISTICS,
        SECURITY_REGION_CODE,
        SECURITY_REGION_NAME,
        TESTED_WITH_RESULT,
        TESTED_POSITIVE,
        @DateTimeNow as Date_Last_Inserted
    FROM
       VWSSTAGE.GGD_TESTED_PEOPLE
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.GGD_TESTED_PEOPLE)

INSERT INTO VWSINTER.GGD_TESTED_PEOPLE
    (
       [VERSION],
       DATE_OF_REPORT,
       DATE_OF_STATISTICS,
       SECURITY_REGION_CODE,
       SECURITY_REGION_NAME,
       TESTED_WITH_RESULT,
       TESTED_POSITIVE,
       Date_Last_Inserted
    )
    SELECT
        [VERSION],
        DATE_OF_REPORT,
        DATE_OF_STATISTICS,
        SECURITY_REGION_CODE,
        SECURITY_REGION_NAME,
        TESTED_WITH_RESULT,
        TESTED_POSITIVE,
        @DateTimeNow as Date_Last_Inserted
    FROM
       VWSSTATIC.GGD_TESTED_PEOPLE
    WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.GGD_TESTED_PEOPLE)

END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_NURSING_HOMES_COMBINED'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_NURSING_HOMES_COMBINED](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'RIVM_NURSING_HOMES_COMBINED'))
BEGIN

	CREATE TABLE [VWSSTAGE].[RIVM_NURSING_HOMES_COMBINED](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_NURSING_HOMES_COMBINED_INTER]
AS
BEGIN
-- Main select and insert into statement. Filtered values on max datelastinserted.
-- Move nursery intake data from staging to intermediate table. 

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_NURSING_HOMES_COMBINED
    (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTIC_REPORTED],
        [SECURITY_REGION_CODE],
        [SECURITY_REGION_NAME],
        [TOTAL_CASES_REPORTED],
        [TOTAL_DECEASED_REPORTED],
        [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        [TOTAL_INFECTED_LOCATIONS_REPORTED],
        [DATE_LAST_INSERTED]
    )
    SELECT
        -- Provided datetime format in source file is yyyy-mm-dd
        CONVERT(DATETIME, [DATE_OF_REPORT], 102),
        CONVERT(DATETIME, [DATE_OF_STATISTIC_REPORTED], 102),
        TRIM([SECURITY_REGION_CODE]),
        [SECURITY_REGION_NAME],
        ISNULL(TOTAL_CASES_REPORTED,0) AS [TOTAL_CASES_REPORTED],
        ISNULL(TOTAL_DECEASED_REPORTED,0) AS [TOTAL_DECEASED_REPORTED],
        ISNULL(TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        ISNULL(TOTAL_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_INFECTED_LOCATIONS_REPORTED],
        @DateTimeNow as [DATE_LAST_INSERTED]
    FROM VWSSTAGE.RIVM_NURSING_HOMES_COMBINED
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.RIVM_NURSING_HOMES_COMBINED)


INSERT INTO VWSINTER.RIVM_NURSING_HOMES_COMBINED
    (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTIC_REPORTED],
        [SECURITY_REGION_CODE],
        [SECURITY_REGION_NAME],
        [TOTAL_CASES_REPORTED],
        [TOTAL_DECEASED_REPORTED],
        [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        [TOTAL_INFECTED_LOCATIONS_REPORTED],
        [DATE_LAST_INSERTED]
    )
    SELECT
        -- Provided datetime format in source file is yyyy-mm-dd
        CONVERT(DATETIME, [DATE_OF_REPORT], 102),
        CONVERT(DATETIME, [DATE_OF_STATISTIC_REPORTED], 102),
        TRIM([SECURITY_REGION_CODE]),
        [SECURITY_REGION_NAME],
        ISNULL(TOTAL_CASES_REPORTED,0) AS [TOTAL_CASES_REPORTED],
        ISNULL(TOTAL_DECEASED_REPORTED,0) AS [TOTAL_DECEASED_REPORTED],
        ISNULL(TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        ISNULL(TOTAL_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_INFECTED_LOCATIONS_REPORTED],
        @DateTimeNow as [DATE_LAST_INSERTED]
    FROM VWSSTATIC.RIVM_NURSING_HOMES_COMBINED
        WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.RIVM_NURSING_HOMES_COMBINED)



END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_REPRODUCTION_NUMBER'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_REPRODUCTION_NUMBER](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE] [varchar](100) NULL,
		[RT_LOW] [varchar](100) NULL,
		[RT_AVG] [varchar](100) NULL,
		[RT_UP] [varchar](100) NULL,
		[POPULATION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate(),
		[VERSION] [varchar](20) NULL
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'RIVM_REPRODUCTION_NUMBER'))
BEGIN

	CREATE TABLE [VWSSTAGE].[RIVM_REPRODUCTION_NUMBER](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE] [varchar](100) NULL,
		[RT_LOW] [varchar](100) NULL,
		[RT_AVG] [varchar](100) NULL,
		[RT_UP] [varchar](100) NULL,
		[POPULATION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate(),
		[VERSION] [varchar](20) NULL
	) ON [PRIMARY]
END

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- Move reproduction data from staging to intermediate table.
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_REPRODUCTION_NUMBER_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_REPRODUCTION_NUMBER
        ([DATE],
        RT_LOW,
        RT_AVG,
        RT_UP,
        DATE_LAST_INSERTED )
SELECT 
        [DATE],
        (CASE WHEN RT_LOW = '' THEN NULL ELSE RT_LOW END) AS RT_LOW,
        (CASE WHEN RT_AVG = '' THEN NULL ELSE RT_AVG END) AS RT_AVG,
        (CASE WHEN RT_UP = '' THEN NULL ELSE RT_UP END) AS RT_UP,
        @DateTimeNow as Date_Last_Inserted
    FROM 
       VWSSTAGE.RIVM_REPRODUCTION_NUMBER
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.RIVM_REPRODUCTION_NUMBER)


INSERT INTO VWSINTER.RIVM_REPRODUCTION_NUMBER
        ([DATE],
        RT_LOW,
        RT_AVG,
        RT_UP,
        DATE_LAST_INSERTED )
SELECT 
        [DATE],
        (CASE WHEN RT_LOW = '' THEN NULL ELSE RT_LOW END) AS RT_LOW,
        (CASE WHEN RT_AVG = '' THEN NULL ELSE RT_AVG END) AS RT_AVG,
        (CASE WHEN RT_UP = '' THEN NULL ELSE RT_UP END) AS RT_UP,
        @DateTimeNow as Date_Last_Inserted
    FROM 
       VWSSTATIC.RIVM_REPRODUCTION_NUMBER
    WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.RIVM_REPRODUCTION_NUMBER)

END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_IC_OPNAME'))
BEGIN


	CREATE TABLE [VWSSTATIC].[RIVM_IC_OPNAME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'RIVM_IC_OPNAME'))
BEGIN


	CREATE TABLE [VWSSTAGE].[RIVM_IC_OPNAME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_IC_OPNAME_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_IC_OPNAME(
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    Date_Last_Inserted 
    )
SELECT
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    @DateTimeNow as Date_Last_Inserted 
FROM 
        VWSSTAGE.RIVM_IC_OPNAME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.RIVM_IC_OPNAME)


INSERT INTO VWSINTER.RIVM_IC_OPNAME(
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    Date_Last_Inserted 
    )
SELECT
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    @DateTimeNow as Date_Last_Inserted 
FROM 
        VWSSTATIC.RIVM_IC_OPNAME
WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.RIVM_IC_OPNAME)


END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'NICE_HOSPITAL'))
BEGIN

	CREATE TABLE [VWSSTATIC].[NICE_HOSPITAL](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[MUNICIPALITY_CODE] [varchar](100) NULL,
		[MUNICIPALITY_NAME] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[HOSPITAL_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS_WEEK_START] [varchar](100) NULL,
		[AGE_GROUP] [varchar](100) NULL,
		[HOSPITAL_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTAGE' 
                 AND  TABLE_NAME = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP'))
BEGIN

	CREATE TABLE [VWSSTAGE].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS_WEEK_START] [varchar](100) NULL,
		[AGE_GROUP] [varchar](100) NULL,
		[HOSPITAL_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO 
        VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
            [VERSION]
            ,[DATE_OF_REPORT]
            ,[DATE_OF_STATISTICS_WEEK_START]
            ,[AGE_GROUP]
            ,[HOSPITAL_ADMISSION_NOTIFICATION]
            ,[HOSPITAL_ADMISSION]
            ,[IC_ADMISSION_NOTIFICATION]
            ,[IC_ADMISSION]
            ,[DATE_LAST_INSERTED]
    )
   SELECT
         [VERSION]
        ,CAST([DATE_OF_REPORT] AS DATETIME)
        ,CAST([DATE_OF_STATISTICS_WEEK_START] AS DATE)
        ,[AGE_GROUP]
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION] AS INT)
		END
       ,@DateTimeNow as Date_Last_Inserted
    FROM 
        VWSSTAGE.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE 
        DATE_LAST_INSERTED = (
            SELECT MAX(DATE_LAST_INSERTED) 
            FROM VWSSTAGE.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)



INSERT INTO 
        VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
            [VERSION]
            ,[DATE_OF_REPORT]
            ,[DATE_OF_STATISTICS_WEEK_START]
            ,[AGE_GROUP]
            ,[HOSPITAL_ADMISSION_NOTIFICATION]
            ,[HOSPITAL_ADMISSION]
            ,[IC_ADMISSION_NOTIFICATION]
            ,[IC_ADMISSION]
            ,[DATE_LAST_INSERTED]
    )
   SELECT
         [VERSION]
        ,CAST([DATE_OF_REPORT] AS DATETIME)
        ,CAST([DATE_OF_STATISTICS_WEEK_START] AS DATE)
        ,[AGE_GROUP]
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION] AS INT)
		END
       ,@DateTimeNow as Date_Last_Inserted
    FROM 
        VWSSTATIC.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE CAST(DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM VWSSTATIC.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)


END;
GO

## Source table for statics

In [ ]:
IF (EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'VWSSTATIC' AND  TABLE_NAME = 'SPLIT_FILE_SOURCES'))
	DROP TABLE [VWSSTATIC].[SPLIT_FILE_SOURCES]

CREATE TABLE [VWSSTATIC].[SPLIT_FILE_SOURCES](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[SOURCE_ID][INT] NULL,
	[BASE_URL] NVARCHAR (MAX) NOT NULL,
	[DOCUMENT_NAME] NVARCHAR (MAX) NOT NULL,
	[TABLE_SCHEMA] NVARCHAR (MAX) NOT NULL,
	[TABLE_NAME] NVARCHAR (MAX) NOT NULL,
	[SOURCE_COLUMNS] NVARCHAR (MAX) NOT NULL, --Get from sources later
	[TARGET_COLUMNS] NVARCHAR (MAX) NOT NULL
) ON [PRIMARY]


In [ ]:
CREATE  OR ALTER VIEW [dbo].[V_STATIC_FILES] AS 
SELECT  
    s.DOCUMENT_NAME as DocumentName,
    s.BASE_URL AS BaseUrl,
    s.TABLE_NAME AS TableName,
    s.TABLE_SCHEMA AS TableSchema,
    dbo.Create_Adf_Mapping(s.[SOURCE_COLUMNS], s.[TARGET_COLUMNS]) AS Mapping
  FROM [VWSSTATIC].[SPLIT_FILE_SOURCES] s
/*
CREATE  OR ALTER VIEW [dbo].[V_STATIC_FILES] AS 
SELECT  
    'COVID-19_aantallen_gemeente_per_dag.csv' as DocumentName,
    'https://data.rivm.nl/covid-19/' AS BaseUrl,
    'RIVM_COVID_19_NUMBER_MUNICIPALITY' AS TableName,
    'VWSSTATIC' AS TableSchema,
    dbo.CreateAdfMapping(s.[SOURCE_COLUMNS], s.[TARGET_COLUMNS]) AS Mapping
  FROM [DATATINO_ORCHESTRATOR_1].[SOURCES] s
  INNER JOIN [VWSSTATIC].[SPLIT_FILE_SOURCES] sfs ON s.Id = sfs.source_id
*/
GO

Nu de instructies toevoegen voor Datatino. Die bestaan uit het toevoegen van een source, workflow, process en twee dataflows. In de database zitten procedures om dit te doen: Upsert\_Source, Upsert\_Workflow en Upsert\_Process. Om dit compact te kunnen doen wordt hier eerst een nieuwe Stored Procedure aangemaakt die dit doet voor één nieuwe statische bron (dbo.Add\_Static\_File). Daarna wordt deze procedure 9 keer aangeroepen met de correcte parameters en uiteindelijk wordt de stored procedure weer uit de database verwijderd.

In [ ]:
DROP PROCEDURE IF EXISTS dbo.Add_Static_File;
GO

In [ ]:



CREATE PROCEDURE dbo.Add_Static_File (@Source_content_url nvarchar(200)
                                        , @Source_content_document nvarchar(200)
                                        , @Workflow_name nvarchar(200)
                                        , @Target_name_schema nvarchar(200)
                                        , @Target_name_table nvarchar(200)
                                        , @source_type nvarchar(200)
                                        , @ColumnMappingSource nvarchar(200)
                                        , @ColumnMappingTarget nvarchar(200))
AS
BEGIN




----------------------------------------------------

    DECLARE @workflow_id INT,        
            @workflow_description VARCHAR(256),
            @is_active INT,
            @Source_content NVARCHAR(400),
            @Target_name NVARCHAR(400);

    SET @Source_content = @Source_content_url + @Source_content_document;
    SET @Target_name = @Target_name_schema + '.' + @Target_name_table

    IF NOT EXISTS (SELECT Id FROM [VWSSTATIC].[SPLIT_FILE_SOURCES] 
                        WHERE [BASE_URL] = @Source_content_url AND [DOCUMENT_NAME] =  @Source_content_document)
        INSERT INTO [VWSSTATIC].[SPLIT_FILE_SOURCES] ([BASE_URL], [DOCUMENT_NAME], [TABLE_SCHEMA], [TABLE_NAME], [SOURCE_COLUMNS], [TARGET_COLUMNS]) 
        VALUES (@Source_content_url, @Source_content_document, @Target_name_schema, @Target_name_table, @ColumnMappingSource, @ColumnMappingTarget)

    /*
    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 0
        WHEN 'acceptance' THEN 0
        ELSE 0
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @SCHEDULE = '0 1 * * FRI', -- AT 01:00 AM, ONLY ON FRIDAY
        @active = @is_active;
    
    DECLARE @source NVARCHAR(512),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    SET @source = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN @Source_content
        WHEN 'acceptance' THEN @Source_content
        ELSE @Source_content
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'Web'
        WHEN 'acceptance' THEN 'Web'
        ELSE 'Web'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'RIVM'
        WHEN 'acceptance' THEN 'RIVM'
        ELSE 'RIVM'
    END;

    SET @source_name = CONCAT('SOURCE_', @workflow_name);
    SET @source_description = CONCAT('LOAD: ', @workflow_name);
    
    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = @ColumnMappingSource,
        @target_columns =  @ColumnMappingTarget,
        @target_name = @target_name,
        @source_type = @source_type,
        @location_type = @location_type,
        @delimiter_type = 'SemiColon', --N/A for json?
        @security_profile= @security_profile;


    --Add to table static sources
    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    IF NOT EXISTS (SELECT Id FROM [VWSSTATIC].[SPLIT_FILE_SOURCES] WHERE [SOURCE_ID] = @source_id)
        INSERT INTO [VWSSTATIC].[SPLIT_FILE_SOURCES] ([SOURCE_ID], [BASE_URL], [DOCUMENT_NAME]) VALUES (@source_id, @Source_content_url, @Source_content_document)
    
    -- 3) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = CONCAT('PROCESS_', @workflow_name, '_TO_STAGE');
    SET @process_description = CONCAT('LOAD: ', @workflow_name);
    SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

        */
END

GO




In [ ]:

Begin TRANSACTION
    DECLARE @Source_content_url_ nvarchar(200) = 'https://data.rivm.nl/data/vws/covid-19/'
    DECLARE @Source_content nvarchar(200) = 'COVID-19_aantallen_gemeente_per_dag_tm_03102021.csv' 
    DECLARE @Workflow_Name nvarchar(200) = 'RIVM_COVID_19_NUMBER_MUNICIPALITY_STATIC_FILE'
    DECLARE @Target_Name nvarchar(200) = 'RIVM_COVID_19_NUMBER_MUNICIPALITY'
    DECLARE @SourceType nvarchar(200) = 'CsvFile'
    DECLARE @ColumnMappingSource nvarchar(200) = 'Date_of_report|Date_of_publication|Municipality_code|Municipality_name|Province|Security_region_code|Security_region_name|Municipal_health_service|ROAZ_region|Total_reported|Deceased'
    DECLARE @ColumnMappingTarget nvarchar(200) = 'DATE_OF_REPORT|DATE_OF_PUBLICATION|MUNICIPALITY_CODE|MUNICIPALITY_NAME|PROVINCE|SECURITY_REGION_CODE|SECURITY_REGION_NAME|MUNICIPAL_HEALTH_SERVICE|ROAZ_REGION|TOTAL_REPORTED|DECEASED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  

    SET @Source_content = 'COVID-19_gehandicaptenzorg_tm_03102021.csv' 
    SET @Workflow_Name = 'DISABILITY_STATIC_FILE'
    SET @Target_Name = 'DISABILITY'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Security_region_name|Total_cases_reported|Total_deceased_reported|Total_new_infected_locations_reported|Total_infected_locations_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|SECURITY_REGION_NAME|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|TOTAL_NEW_INFECTED_LOCATIONS_REPORTED|TOTAL_INFECTED_LOCATIONS_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_thuiswonend_70plus_tm_03102021.csv' 
    SET @Workflow_Name = 'ELDERLY_AT_HOME_STATIC_FILE'
    SET @Target_Name = 'ELDERLY_AT_HOME'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Security_region|Total_cases_reported|Total_deceased_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|SECURITY_REGION|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_uitgevoerde_testen_tm_03102021.csv' 
    SET @Workflow_Name = 'GGD_TESTED_PEOPLE_STATIC_FILE'
    SET @Target_Name = 'GGD_TESTED_PEOPLE'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|Security_region_code|Security_region_name|Tested_with_result|Tested_positive'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|SECURITY_REGION_CODE|SECURITY_REGION_NAME|TESTED_WITH_RESULT|TESTED_POSITIVE|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_verpleeghuizen_tm_03102021.csv' 
    SET @Workflow_Name = 'RIVM_NURSING_HOMES_COMBINED_STATIC_FILE'
    SET @Target_Name = 'RIVM_NURSING_HOMES_COMBINED'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Total_cases_reported|Total_deceased_reported|Total_new_infected_locations_reported|Total_infected_locations_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|TOTAL_NEW_INFECTED_LOCATIONS_REPORTED|TOTAL_INFECTED_LOCATIONS_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_reproductiegetal_tm_03102021.json' 
    SET @Workflow_Name = 'RIVM_REPRODUCTION_NUMBER_STATIC_FILE'
    SET @Target_Name = 'RIVM_REPRODUCTION_NUMBER'
    SET @SourceType = 'JsonFile'
    SET @ColumnMappingSource = 'Date|Rt_low|Rt_avg|Rt_up|population'
    SET @ColumnMappingTarget = 'DATE|RT_LOW|RT_AVG|RT_UP|POPULATION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_ic_opnames_tm_03102021.csv' 
    SET @Workflow_Name = 'RIVM_IC_OPNAME_STATIC_FILE'
    SET @Target_Name = 'RIVM_IC_OPNAME'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|IC_admission_notification|IC_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|IC_ADMISSION_NOTIFICATION|IC_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_ziekenhuisopnames_tm_03102021.csv' 
    SET @Workflow_Name = 'NICE_HOSPITAL_STATIC_FILE'
    SET @Target_Name = 'NICE_HOSPITAL'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|Municipality_code|Municipality_name|Security_region_code|Security_region_name|Hospital_admission_notification|Hospital_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|MUNICIPALITY_CODE|MUNICIPALITY_NAME|SECURITY_REGION_CODE|SECURITY_REGION_NAME|HOSPITAL_ADMISSION_NOTIFICATION|HOSPITAL_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'COVID-19_ziekenhuis_ic_opnames_per_leeftijdsgroep_tm_03102021.csv' 
    SET @Workflow_Name = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_STATIC_FILE'
    SET @Target_Name = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics_week_start|Age_group|Hospital_admission_notification|Hospital_admission|IC_admission_notification|IC_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS_WEEK_START|AGE_GROUP|HOSPITAL_ADMISSION_NOTIFICATION|HOSPITAL_ADMISSION|IC_ADMISSION_NOTIFICATION|IC_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content_url_, @Source_content, @Workflow_Name, 'VWSSTATIC', @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  

COMMIT


DROP PROCEDURE IF EXISTS dbo.Add_Static_File;
GO